In [21]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
import os

In [2]:
df = pd.read_csv('train_set.csv')
df

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry
...,...,...,...
28816,9969,../data/images/train/surprise\9969.jpg,surprise
28817,9985,../data/images/train/surprise\9985.jpg,surprise
28818,9990,../data/images/train/surprise\9990.jpg,surprise
28819,9992,../data/images/train/surprise\9992.jpg,surprise


In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['target'] = label_encoder.fit_transform(df['label'])

In [23]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')

def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, 0)
    smallimage = cv2.resize(image, (32, 32))
    image_array = np.array(smallimage)
    return image_array

df['imagenes'] =df['path'].apply(cargar_y_convertir_imagen)

In [8]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [44]:
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(32, 32, 1))

X = np.stack(df['imagenes'].to_numpy())
y = df['target']

X = X / 255.0 


In [45]:
from keras.callbacks import ModelCheckpoint
mcheckpoint = ModelCheckpoint("callback_model.h5")

In [46]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(patience=5)

In [47]:
modelo_cnn = Sequential()
modelo_cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Conv2D(128, (3, 3), activation='relu'))
modelo_cnn.add(MaxPooling2D((2, 2)))
modelo_cnn.add(Flatten())
modelo_cnn.add(Dense(128, activation='relu'))
modelo_cnn.add(Dense(64, activation='relu'))
modelo_cnn.add(Dense(7, activation='sigmoid'))

modelo_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

modelo_cnn.summary()

modelo_cnn.fit(X, y, epochs=50, batch_size=32, validation_split=0.2,callbacks = [earlystop, mcheckpoint])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 30, 30, 64)        640       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 15, 15, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 6, 6, 128)         0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 4608)              0         
                                                                 
 dense_12 (Dense)            (None, 128)              

In [48]:
from tensorflow.keras.models import save_model

modelo_cnn.save('modelo_cnn.h5')

In [49]:
test=pd.read_csv('test_set.csv')

In [50]:
test['path'] = test['id_img'].apply(lambda x: f'./data/images/test/{x}.jpg')

In [51]:
test['imagenes'] = test['path'].apply(cargar_y_convertir_imagen)

In [53]:
test.head()

,id_img,path,imagenes
0,10052,./data/images/test/10052.jpg,"[[61, 69, 92, 141, 123, 79, 70, 76, 84, 84, 11..."
1,10065,./data/images/test/10065.jpg,"[[25, 22, 8, 19, 28, 12, 77, 83, 39, 43, 39, 2..."
2,10079,./data/images/test/10079.jpg,"[[196, 182, 190, 203, 205, 222, 229, 231, 231,..."
3,10095,./data/images/test/10095.jpg,"[[87, 77, 76, 86, 100, 99, 108, 123, 149, 160,..."
4,10121,./data/images/test/10121.jpg,"[[8, 2, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 1,..."


In [54]:
from tensorflow.keras.models import load_model

modelo_cnn = load_model('modelo_cnn.h5')  

test['imagenes'] = test['imagenes'].apply(lambda x: np.array(x).reshape(32, 32, 1))

test['imagenes'] = test['imagenes'] / 255.0

X_test = np.stack(test['imagenes'].to_numpy())
predicciones = modelo_cnn.predict(X_test)

predicciones_labels = np.argmax(predicciones, axis=1)

test['label_encoded_pred'] = predicciones_labels

221/221 [==============================] - 1s 5ms/step


In [55]:
test.drop(columns=(['path', 'imagenes']), inplace=True)

In [56]:
test['label'] = label_encoder.inverse_transform(predicciones_labels)

In [57]:
submission = pd.DataFrame({'id_img': test['id_img'], 'label': test['label']})

submission.to_csv('submission.csv', index=False)

In [58]:
submission

,id_img,label
0,10052,angry
1,10065,sad
2,10079,angry
3,10095,angry
4,10121,angry
...,...,...
7061,9806,surprise
7062,9830,happy
7063,9853,surprise
7064,9878,fear
